We should have our scaled train and test data. 

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import StandardScaler

In [ ]:
# Save model as JSON
nn_json = nn.to_json()

file_path = ("../Resources/model.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

In [ ]:
# Save weights
file_path = "../Resources/model.h5"
nn.save_weights(file_path)

### Intro to transfer learning

Say we wanted to use a previous model and train it with more data, without having to run the whole model over again. Consider if it took days to train our model? We would want to save that and then add to it. The following is how we would do so:

In [ ]:
#Imports
from tensorflow.keras.models import model_from_json

# load json and create model
file_path = ("../Resources/model.json")
with open("../Resources/model.json", "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = "../Resources/model.h5"
loaded_model.load_weights(file_path)

In [ ]:
# The `layers` attribute stores the various layers in the model as a list
loaded_model.layers

### Load in new data but work with OLD the loaded model.

In [ ]:
# Load in some new data
X = pd.read_csv('../Resources/meet_or_beat_US_new_data.csv')
X.head()

In [ ]:
new_predictions = loaded_model.predict(X)
new_predictions

In [ ]:
new_data = pd.read_csv('../Resources/meet_or_beat_AU.csv')
new_data.head()

In [ ]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split

y_var = 'after_total_returns'
x_vars = list(new_data.columns)
x_vars.remove(y_var)

X_train, X_test, y_train, y_test = train_test_split(new_data[x_vars], new_data[y_var], random_state=1)

# Create the StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Freeze layers

In [ ]:
# Freeze the existing layers of the loaded model
for layer in loaded_model.layers[0:-1]:
    layer.trainable = False

In [ ]:
loaded_model.summary()

### Now we can add new layers to existing model

In [ ]:
# Create a new DNN to hold the old one
transfer_model = Sequential()
# Go through each layer, skipping the last layer
for layer in loaded_model.layers[:-1]: 
    transfer_model.add(layer)

In [ ]:
transfer_model.summary()

In [ ]:
# Add an additional layer
transfer_model.add(Dense(10, activation="relu"))
# Add the final output layer
transfer_model.add(Dense(1))

In [ ]:
transfer_model.summary()

### Fit revised model

In [ ]:
# Compile the Sequential model
transfer_model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model
transfer_model.fit(X_train_scaled,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")